In [21]:
import requests
import pandas as pd
import os
import time
from openpyxl import load_workbook

In [22]:
def save_data_to_excel(data):
    df = pd.DataFrame(data)
    filename = f'instagram_users_2.xlsx'
    
    try:
        # 기존 파일이 있고 손상되었는지 확인
        if os.path.exists(filename):
            try:
                existing_df = pd.read_excel(filename, sheet_name="Data")
            except Exception as e:
                print(f"기존 파일이 손상되었습니다. 새로운 파일을 생성합니다: {e}")
                existing_df = pd.DataFrame({col: pd.Series(dtype=str) for col in df.columns})
        else:
            existing_df = pd.DataFrame({col: pd.Series(dtype=str) for col in df.columns})

        # 데이터 타입 변환
        df = df.astype(str)
        existing_df = existing_df.astype(str)

        # 중복 제거 (User ID가 있는 경우)
        if "ID" in df.columns and "ID" in existing_df.columns:
            existing_ids = set(existing_df["ID"])
            df = df[~df["ID"].isin(existing_ids)]

        # 새로운 데이터 추가 - 이 부분이 핵심 수정 부분입니다
        if not df.empty:
            final_df = pd.concat([existing_df, df], ignore_index=True)
            with pd.ExcelWriter(filename, engine='openpyxl', mode='w') as writer:
                final_df.to_excel(writer, sheet_name="Data", index=False)
            print(f"{len(df)} new rows added to {filename}.")
        else:
            print("No new data to add. All entries are duplicates.")

    except FileNotFoundError:
        # 파일이 없을 경우 새로운 파일 생성
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name="Data")

In [23]:
def fetch_and_save_user_data(user_name, api_key):
    """
    개별 인스타그램 게시물의 좋아요 사용자 정보를 가져와 엑셀 파일로 저장하는 함수입니다.
    count 파라미터로 가져올 사용자 수를 지정할 수 있습니다.
    """

    # RapidAPI URL 및 키 설정
    url = "https://instagram-scraper-api2.p.rapidapi.com/v1/info"
    querystring = {"username_or_id_or_url": user_name}
    headers = {
        "x-rapidapi-key": api_key,  # API 키를 인자로 받음
        "x-rapidapi-host": "instagram-scraper-api2.p.rapidapi.com"
    }

    # API 요청
    response = requests.get(url, headers=headers, params=querystring)

    # 응답 처리
    if response.status_code == 200:
        data = response.json().get("data", {})

        # 개별 좋아요 사용자 정보 저장
        instagram_likers = data.get("instagram_likers", {}).get("edges", [])
        user_data = [{
            "Username": data.get("username"),
            "ID": data.get("id"),
            "biography": data.get("biography"),
            "follower_count": data.get("follower_count"),
            "following_count": data.get("following_count"),
            "media_count": data.get("media_count"),
        }]


        # DataFrame으로 변환 및 엑셀 파일로 저장
        save_data_to_excel(user_data)  # 데이터를 중간 저장
        print(f"사용자 '{user_name}'의 정보가 저장되었습니다.")
    else:
        print(f"Failed to fetch data for {user_name}. Status code: {response.status_code}")

In [24]:
def process_posts_from_file(file_path, api_key):
    """
    CSV 또는 엑셀 파일에서 게시물 URL을 불러와 각 게시물의 좋아요 사용자 정보를 저장하는 함수입니다.
    count 파라미터로 가져올 사용자 수를 지정할 수 있습니다.
    """
    # 파일 확장자에 따라 CSV 또는 엑셀 파일로 불러오기
    if file_path.endswith('.csv'):
        posts_df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        posts_df = pd.read_excel(file_path)
    else:
        print("지원되지 않는 파일 형식입니다. CSV 또는 엑셀 파일을 사용하세요.")
        return

    # 'post_url' 열이 있어야 함
    if 'user_name' not in posts_df.columns:
        print("파일에 'user_name' 열이 포함되어 있어야 합니다.")
        return

    # 각 게시물 URL에 대해 좋아요 사용자 정보를 수집하고 저장

    for _, row in posts_df.iterrows():
        user_name = row['user_name']
        fetch_and_save_user_data(user_name, api_key)
        #fetch_and_save_likes_data2(post_url, api_key, count=count)

In [25]:
api_key = "fb1be9caf7mshc1fea79903f370fp1c0b11jsn97070d82933d"  # 본인의 RapidAPI 키로 교체
file_path = "users_list_1.xlsx"  # 게시물 URL 목록이 포함된 파일 경로
process_posts_from_file(file_path, api_key)  # 원하는 count 값으로 지정

No new data to add. All entries are duplicates.
사용자 'bami_84'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'es_home2'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'tirtir_official'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 '29_stay'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'leesj2281'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'heeddo.fit'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'i.m.suhyun'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'tirlife.official'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 '_yo___jini2'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'seonho__kim'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'k_dorimm'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.
사용자 'the__naeun___'의 정보가 저장되었습니다.
No new data to add. All entries are duplicates.

KeyboardInterrupt: 